## SK 데이터 허브 Rest API

get_access 모듈을 통해 획득한 이용권한과 pid 파라미터를 통해 데이터를 Rest API방식으로 자동으로 수집하고, 수집된 Json파일을 csv로 정재한다.

In [2]:
import json
import os
import time
from urllib.request import urlopen
from pandas.io.json import json_normalize
from get_access import get_access

In [78]:
pids = get_access('your id', 'your pw', '치킨')

In [80]:
directory = "../data/"
token = 'your token key'

df = pd.DataFrame(columns=['성별','시군구','시도','업종','연령대','요일','통화건수','년','월','일'])
i = 0

start_time = time.time()
for num, pid in enumerate(pids):
    page = 1
    info_url = 'https://api.bigdatahub.co.kr/v1/datahub/datasets/datainfo.json?pid={0}&TDCAccessKey={1}'.format(pid,token)
    con_url = 'https://api.bigdatahub.co.kr/v1/datahub/datasets/search.json?pid={0}&$count=1000&$page={2}&TDCAccessKey={1}'.format(pid,token,page)
    print(pid)
    print(time.time()-start_time)
    print(i, df.shape)
    response_info = urlopen(info_url).read().decode('utf-8')
    response_con = urlopen(con_url).read().decode('utf-8')
    page_max = round(json.loads(response_con)['totalResult']/1000)
    
    json_f = json.loads(response_con)
    csv = json_normalize(json_f['entry'])
    csv['년'] = csv['기준일'].apply(lambda x : x[:4])
    csv['월'] = csv['기준일'].apply(lambda x : x[4:6])
    csv['일'] =  csv['기준일'].apply(lambda x : x[6:8])
    csv.drop('기준일', axis=1, inplace=True)
    
    df = pd.concat([df,csv], axis=0)
    i += 1 
    with open(directory + json.loads(response_info)['productName'] + '_=' + str(page) + '.json', 'w', encoding='utf-8') as f:
        f.write(response_con)
    
    for page in range(1,page_max+1):
        # print(page)
        con_url = 'https://api.bigdatahub.co.kr/v1/datahub/datasets/search.json?pid={0}&$count=1000&$page={2}&TDCAccessKey={1}'.format(pid,token,page)
        response_con = urlopen(con_url).read().decode('utf-8')
        
        json_f = json.loads(response_con)
        csv = json_normalize(json_f['entry'])
        csv['년'] = csv['기준일'].apply(lambda x : x[:4])
        csv['월'] = csv['기준일'].apply(lambda x : x[4:6])
        csv['일'] =  csv['기준일'].apply(lambda x : x[6:8])
        csv.drop('기준일', axis=1, inplace=True)
        df = pd.concat([df,csv], axis=0)
        i += 1 
        print(time.time()-start_time)
        print(i, df.shape)
        with open(directory + json.loads(response_info)['productName'] +'_=' + str(page) +'.json', 'w', encoding='utf-8') as f:
            f.write(response_con)
            
        
        
       

1001463
0.000637054443359375
0 (0, 10)
3.8597989082336426
0 (2000, 12)
5.845602035522461
0 (3000, 12)
10.304686069488525
0 (4000, 12)
11.367753028869629
0 (5000, 12)
12.463568925857544
0 (6000, 12)
15.634541034698486
0 (7000, 12)
17.392119884490967
0 (8000, 12)
20.460668087005615
0 (9000, 12)
22.799230098724365
0 (10000, 12)
23.635917901992798
0 (11000, 12)
26.202392101287842
0 (12000, 12)
26.95658588409424
0 (13000, 12)
29.617584943771362
0 (14000, 12)
30.092427015304565
0 (15000, 12)
30.660999059677124
0 (16000, 12)
33.37177395820618
0 (17000, 12)
34.187126874923706
0 (18000, 12)
34.77867293357849
0 (19000, 12)
37.632688999176025
0 (20000, 12)
40.42811608314514
0 (21000, 12)
41.124666929244995
0 (22000, 12)
44.88860106468201
0 (23000, 12)
46.62205100059509
0 (24000, 12)
49.409971952438354
0 (25000, 12)
52.2092399597168
0 (26000, 12)
54.956352949142456
0 (27000, 12)
55.629723072052
0 (28000, 12)
59.46386003494263
0 (29000, 12)
62.52674198150635
0 (30000, 12)
63.19867491722107
0 (31000

KeyboardInterrupt: 

In [82]:
df.head()

,년,성별,시군구,시도,업종,연령대,요일,월,읍면동,이용건수,일,통화건수
0,2017,남,강남구,서울특별시,치킨,40대,일,01,개포동,NaN,01,9
1,2017,남,강남구,서울특별시,치킨,60대이상,일,01,세곡동,NaN,01,5
2,2017,여,강남구,서울특별시,치킨,40대,일,01,역삼동,NaN,01,36
3,2017,여,강남구,서울특별시,치킨,40대,일,01,일원동,NaN,01,12
4,2017,여,강남구,서울특별시,치킨,40대,일,01,도곡동,NaN,01,17
